In [ ]:
%%capture
!pip install pennylane

In [1]:
import numpy as np
import cvxpy as cp
import scipy as sc
import pennylane as qml
import torch
import torch.nn as nn
import torch.nn.functional as Fn
import torch.optim as optim
from torch.autograd import Variable

In [15]:
# quantum circuit settings
num_wires = 4
num_layers = 3
num_shots = 1
num_of_samples = 100

# dimension of the system
N = 4

# initiate the quantum device
device = qml.device("default.qubit", wires=num_wires, shots=num_of_samples)

# initialize the parameters of a unitary that we will apply
# to rho and sigma
# For the case of von neumann entropy, sigma is maximally mixed state.
# Therefore, for that, we can just use a uniform number generator for sampling

@qml.qnode(device)
def measure_rho(param):
    qml.Hadamard(wires=0)
    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[1, 2])
    qml.CNOT(wires=[2, 3])
    qml.PauliZ(wires=0)
    qml.PauliX(wires=0)
    qml.PauliY(wires=1)

    # Apply the parameterized unitary to it
    qml.RandomLayers(param, wires=[0, 1])
#     return qml.density_matrix(wires=[0, 1])

    # measure in the computational basis
    result = qml.sample(qml.PauliZ(0)), qml.sample(qml.PauliZ(1))
    return result

# maximally mixed state
sigma = np.eye(N)/N

In [ ]:
param_init = np.random.random(qml.RandomLayers.shape(n_layers=num_layers, n_rotations=3))
rho_test = measure_rho(param_init)
print(rho_test)

In [ ]:
# Evaluate von Neumann entropy using its analytical forumla

# ignore the division by zero
np.seterr(divide = 'ignore')

# quantum relative entropy
H_rho = np.real(np.trace(rho_test @ (sc.linalg.logm(rho_test) - sc.linalg.logm(sigma))))

np.seterr(divide = 'warn')

# print the result
print(np.log(N)-H_rho)



In [13]:
class neural_function(nn.Module):
    def __init__(self,dimension,hidden_layer):
        super(neural_function, self).__init__()

        self.dimension = dimension
        self.hidden_layer = hidden_layer
        self.lin1 = nn.Linear(self.dimension, self.hidden_layer)
        self.lin_end = nn.Linear(self.hidden_layer, 1)

    def forward(self, input):
        y = torch.sigmoid(self.lin1(input.float()))
        y = self.lin_end(y)

        return y

In [ ]:
#@title Optimization using Gradient Descent (with neural network)

# parameters of the optimization
num_of_epochs = 1000
learning_rate = 0.05
num_of_samples = 100
dimension = 2
hidden_layer = 10

# initialize the neural network and quantum circuit parameters
neural_fn = neural_function(dimension, hidden_layer)
param_init = np.random.random(qml.RandomLayers.shape(n_layers=num_layers, n_rotations=3))

# intialize the cost function store
cost_func_store = []

# start the training
for epoch in range(1, num_of_epochs):

  # evaluate the gradient with respect to the quantum circuit parameters
    gradients = np.zeros_like((param_init))
    for i in range(len(gradients)):
        for j in range(len(gradients[0])):

      # copy the parameters
            shifted = param_init.copy()

      # right shift the parameters
            shifted[i, j] += np.pi/2

      # forward evaluation
            forward_sum = 0
            result = measure_rho(shifted)
            for sample in range(num_of_samples):
                sample_result_array = np.array([result[0][sample], result[1][sample]])
                nn_result = neural_fn(torch.from_numpy(sample_result_array))
                forward_sum += nn_result[0].detach().numpy()

      # normalize the forward sum
            forward_sum = forward_sum/num_of_samples

      # left shift the parameters
            shifted[i, j] -= np.pi

      # backward evaluation
            backward_sum = 0
            result = measure_rho(shifted)
            for sample in range(num_of_samples):
                sample_result_array = np.array([result[0][sample], result[1][sample]])
                nn_result = neural_fn(torch.from_numpy(sample_result_array))
                backward_sum += nn_result[0].detach().numpy()

      # normalize the backward sum
            backward_sum = backward_sum/num_of_samples
      #print(backward_sum)

      # parameter-shift rule
            gradients[i, j] = - 0.5 * (forward_sum - backward_sum)

  # first copy the quantum circuit parameters before updating it
    prev_param_init = param_init.copy()

  # update the quantum circuit parameters
    param_init -= learning_rate*gradients

  # evaluate the gradient with respect to the NN parameters

  # evaluate the first term
    grad_w1 = torch.zeros_like(neural_fn.lin1.weight)
    grad_b1 = torch.zeros_like(neural_fn.lin1.bias)
    grad_w2 = torch.zeros_like(neural_fn.lin_end.weight)
    grad_b2 = torch.zeros_like(neural_fn.lin_end.bias)

    result = measure_rho(prev_param_init)
    for sample in range(num_of_samples):
        sample_result_array = np.array([result[0][sample], result[1][sample]])
        nn_result = neural_fn(torch.from_numpy(sample_result_array))
        nn_result.backward()
        grad_w1 += neural_fn.lin1.weight.grad*(1/num_of_samples)
        grad_b1 += neural_fn.lin1.bias.grad*(1/num_of_samples)
        grad_w2 += neural_fn.lin_end.weight.grad*(1/num_of_samples)
        grad_b2 += neural_fn.lin_end.bias.grad*(1/num_of_samples)
        neural_fn.lin1.weight.grad.zero_()
        neural_fn.lin1.bias.grad.zero_()
        neural_fn.lin_end.weight.grad.zero_()
        neural_fn.lin_end.bias.grad.zero_()

  # evaluate the second term
    grad_w1_2 = torch.zeros_like(neural_fn.lin1.weight.grad)
    grad_b1_2 = torch.zeros_like(neural_fn.lin1.bias.grad)
    grad_w2_2 = torch.zeros_like(neural_fn.lin_end.weight.grad)
    grad_b2_2 = torch.zeros_like(neural_fn.lin_end.bias.grad)

    for sample in range(num_of_samples):
        result = np.random.choice([-1, 1], size=2)
        nn_result = neural_fn(torch.from_numpy(result.flatten()))
        nn_result.backward()
        grad_w1_2 += (np.exp(nn_result[0].detach().numpy()))*neural_fn.lin1.weight.grad*(1/num_of_samples)
        grad_b1_2 += (np.exp(nn_result[0].detach().numpy()))*neural_fn.lin1.bias.grad*(1/num_of_samples)
        grad_w2_2 += (np.exp(nn_result[0].detach().numpy()))*neural_fn.lin_end.weight.grad*(1/num_of_samples)
        grad_b2_2 += (np.exp(nn_result[0].detach().numpy()))*neural_fn.lin_end.bias.grad*(1/num_of_samples)
        neural_fn.lin1.weight.grad.zero_()
        neural_fn.lin1.bias.grad.zero_()
        neural_fn.lin_end.weight.grad.zero_()
        neural_fn.lin_end.bias.grad.zero_()

  # evaluate the difference, i.e., the gradient

    nn_grad_W1 = grad_w1_2 - grad_w1
    nn_grad_b1 = grad_b1_2 - grad_b1
    nn_grad_W2 = grad_w2_2 - grad_w2
    nn_grad_b2 = grad_b2_2 - grad_b2

  # update the NN weights and normalize them
    with torch.no_grad():
        neural_fn.lin1.weight -= learning_rate*nn_grad_W1
        neural_fn.lin1.bias -= learning_rate*nn_grad_b1
        neural_fn.lin_end.weight -= learning_rate*nn_grad_W2
        neural_fn.lin_end.bias -= learning_rate*nn_grad_b2

  # evaluate the cost function at these parameters
    first_term = 0
    result = measure_rho(param_init)
    for sample in range(num_of_samples):
        sample_result_array = np.array([result[0][sample], result[1][sample]])
        nn_result = neural_fn(torch.from_numpy(sample_result_array))
        first_term += nn_result[0].detach().numpy()

  # normalize the cost sum
    first_term = first_term/num_of_samples

  # # Second term evaluation
    second_term = 0
    for sample in range(num_of_samples):
        result = np.random.choice([-1, 1], size=2)
        nn_result = neural_fn(torch.from_numpy(result.flatten()))
        second_term += np.exp(nn_result[0].detach().numpy())

  # normalize the second term sum
    second_term = second_term/num_of_samples

    # add the cost function to the store
    cost_func_store.append(np.log(N) - first_term + second_term - 1)

  # print the cost
    print(np.log(N) - first_term + second_term - 1)

In [ ]:
#@title Optimization using Gradient Descent (without neural network)

# parameters of the optimization
num_of_epochs = 1000
learning_rate = 0.08
num_of_samples = 100
deviation = 1

# initialize the neural network and quantum circuit parameters
W = deviation*np.random.rand(3,3)
np.random.random(qml.RandomLayers.shape(n_layers=num_layers, n_rotations=3))

# intialize the cost function store
cost_func_store = []


# start the training
for epoch in range(1, num_of_epochs):

  # evaluate the gradient with respect to the quantum circuit parameters
    gradients = np.zeros_like((param_init))
    for i in range(len(gradients)):
        for j in range(len(gradients[0])):

      # copy the parameters
            shifted = param_init.copy()

      # right shift the parameters
            shifted[i, j] += np.pi/2

      # forward evaluation
            forward_sum = 0
            result = measure_rho(shifted)
            for sample in range(num_of_samples):
                forward_sum += W[result[0][sample]][result[1][sample]]

      # normalize the forward sum
            forward_sum = forward_sum/num_of_samples

      # left shift the parameters
            shifted[i, j] -= np.pi

      # backward evaluation
            backward_sum = 0
            result = measure_rho(shifted)
            for sample in range(num_of_samples):
                backward_sum += W[result[0][sample]][result[1][sample]]

      # normalize the backward sum
            backward_sum = backward_sum/num_of_samples
      #print(backward_sum)

      # parameter-shift rule
            gradients[i, j] = - 0.5 * (forward_sum - backward_sum)

  # first copy the quantum circuit parameters before updating it
    prev_param_init = param_init.copy()

  # update the quantum circuit parameters
    param_init -= learning_rate*gradients

  # evaluate the gradient with respect to the NN parameters


  # evaluate the gradient with respect to the eigenvalues

  # 1 , 1
    E = np.zeros_like(W)
    E[1][1] = 1
    dW1 = 0
    result = measure_rho(prev_param_init)
    for sample in range(num_of_samples):
        dW1 -= E[result[0][sample]][result[1][sample]]


    # normalize it
    dW1 = dW1/num_of_samples
    dW1 += (np.exp(W[1][1])/N)

    # 1 , -1
    E = np.zeros_like(W)
    E[1][-1] = 1
    dW2 = 0
    result = measure_rho(prev_param_init)
    for sample in range(num_of_samples):
        dW2 -= E[result[0][sample]][result[1][sample]]


    # normalize it
    dW2 = dW2/num_of_samples
    dW2 += (np.exp(W[1][-1])/N)

    # -1 , 1
    E = np.zeros_like(W)
    E[-1][1] = 1
    dW3 = 0
    result = measure_rho(prev_param_init)
    for sample in range(num_of_samples):
        dW3 -= E[result[0][sample]][result[1][sample]]


    # normalize it
    dW3 = dW3/num_of_samples
    dW3 += (np.exp(W[-1][1])/N)

    # -1 , -1
    E = np.zeros_like(W)
    E[-1][-1] = 1
    dW4 = 0
    result = measure_rho(prev_param_init)
    for sample in range(num_of_samples):
        dW4 -= E[result[0][sample]][result[1][sample]]


    # normalize it
    dW4 = dW4/num_of_samples
    dW4 += (np.exp(W[-1][-1])/N)


    # update the eigenvalues
    W[1][1] -= learning_rate*dW1
    W[1][-1] -= learning_rate*dW2
    W[-1][1] -= learning_rate*dW3
    W[-1][-1] -= learning_rate*dW4

    # evaluate the cost function at these parameters
    first_term = 0
    result = measure_rho(param_init)
    for sample in range(num_of_samples):
        first_term += W[result[0][sample]][result[1][sample]]

    # normalize the cost sum
    first_term = first_term/num_of_samples

    # # Second term evaluation
    second_term = np.exp(W[1][1]) + np.exp(W[1][-1]) + np.exp(W[-1][1]) + np.exp(W[-1][-1])

    # normalize the second term sum
    second_term = second_term/N

    # add the cost function to the store
    cost_func_store.append(np.log(N) - first_term + second_term - 1)

    # print the cost
    print(np.log(N) - first_term + second_term - 1)